In [ ]:
# default_exp facade


In [29]:
#export
import random
from nbdev.showdoc import *
import pandas as pd
import sentencepiece as sp
from pathlib import Path
from collections import Counter
from ds4se.mining.unsupervised.traceability.eval import *
# import ds4se.mining.unsupervised.traceability.approach.cisco as cisco
from enum import Enum, unique, auto
from ds4se.exp import i
import os
import pkg_resources
from sklearn.manifold import TSNE
import numpy as np

In [30]:
#export
@unique
class LinkType(Enum):
    req2tc = auto()
    req2src = auto()

In [31]:
#Helper functions for data analysis part of library

#export
"""
Helper functions to extract all entries in the dataframe, which will be contents of either source or target artifacts. Contents are stored
in the column named "contents". This helper function retrive strings stored in this columns and stored them in a list.
:param df: dataframe of content that need to be processed
:param spm: sentence piece processor that will process the dataframe
:returns: documents lists of all entries in the dataframe 
"""
def get_docs(df, spm):
    docs = []
    for fn in df["contents"]:
        docs += spm.EncodeAsPieces(fn)
    return docs

#export
"""
Helper functions to retrive counter object for all tokens in a dataframe. 
:param docs: doc list of contents that we need to extract information on tokens and their frequency
:returns: documents counter of token and corresponding occurrence. 
"""
def get_counters(docs):
    #param doc list of contents that need info on tokens
    #return the counters object of tokens   
    doc_cnts = []
    cnt = Counter()
    for tok in docs:
        cnt[tok] += 1 
        doc_cnts.append(cnt)
    return doc_cnts

#export
"""
Helper functions that get a dataframe and generate a Counter object for all of them tokens it contains, 
as well as their frequency. It load sentence piece model and call two helper function to calculate token freqnency 
:param artifacts_df: doc list of contents that we need to extract information on tokens and their frequency
:returns: counter object with token and term occurrence. 
"""
def preprocess(artifacts_df):
    spm = sp.SentencePieceProcessor()
    bpe_model_path = pkg_resources.resource_filename('ds4se', 'model/test.model')
    spm.Load(bpe_model_path)
    docs = get_docs(artifacts_df,spm)
    cnts = get_counters(docs)
    return cnts

In [32]:
#export
"""
Calculate traceability of two strings of artifacts with given techniques. Method will group two strings as a pair and feed into
the traceability model to get result. If users switch the order of source and target, result should be very similar. 
:param source: a string of the entire source file  
:param target: a string of the entire target file 
:param technique: what tecchnique to use to calculate traceability
:param word2vec_metric: optional, what metric to use to calculate traceability. Only for word2vec
:returns: a tuple: (distance, similarity), similarity is the traceability value
"""
def TraceLinkValue(source, target, technique, word2vec_metric = "WMD"):
    
    dummy_path = pkg_resources.resource_filename('ds4se', 'model/val.csv')

    
    value1 = random.randint(0,100)/100
    value2 = random.randint(0,100)/100
    value = (value1, value2)

    
    if (technique == "VSM"):
        pass
    if (technique == "LDA"):
        pass
    if (technique == "orthogonal"):
        pass
    if (technique == "LSA"):
        pass
    if (technique == "JS"):
        pass
    if (technique == "word2vec"):
        model_path = pkg_resources.resource_filename('ds4se', 'model/word2vec_libest.model')
        parameter = {
            "vectorizationType": VectorizationType.word2vec,
            "linkType": LinkType.req2tc,
            "system": 'libest',
            "path_to_trained_model": model_path,
            "source_path": dummy_path,
            "target_path": dummy_path,
            "system_path": dummy_path,
            "saving_path": 'test_data/',
            "names": ['Source','Target','Linked?']
        }
        
        source_df = pd.DataFrame({ "ids": ["source"],  "text":[source]})
        target_df = pd.DataFrame({ "ids": ["target"],  "text":[target]})
        word2vec = Word2VecSeqVect(parameter)
        word2vec.df_source = source_df
        word2vec.df_target = target_df
        links = [(source_df["ids"][0],target_df["ids"][0])]
        if (word2vec_metric == "SCM"):
            computeDistanceMetric = word2vec.computeDistanceMetric(links, metric_list = [DistanceMetric.SCM])
        else:
            computeDistanceMetric = word2vec.computeDistanceMetric(links, metric_list = [DistanceMetric.WMD])
        value = (computeDistanceMetric[0][0][2],computeDistanceMetric[0][0][3])    
        
    if (technique == "doc2vec"):
        model_path = pkg_resources.resource_filename('ds4se', 'model/doc2vec_libest.model')
        parameter = {
            "vectorizationType": VectorizationType.doc2vec,
            "linkType": LinkType.req2tc,
            "system": 'libest',
            "path_to_trained_model": model_path,
            "source_path": dummy_path,
            "target_path": dummy_path,
            "system_path": dummy_path,
            "saving_path": 'test_data/',
            "names": ['Source','Target','Linked?']
        }
        
        source_df = pd.DataFrame({ "ids": ["source"],  "text":[source]})
        target_df = pd.DataFrame({ "ids": ["target"],  "text":[target]})
        doc2vec = Doc2VecSeqVect(params = parameter)
        doc2vec.df_source = source_df
        doc2vec.df_target = target_df
        links = [(source_df["ids"][0],target_df["ids"][0])]
        doc2vec.InferDoc2Vec(steps=200)
        table = doc2vec.computeDistanceMetric( links, metric_list = [DistanceMetric.EUC] )
        value = (table[0][0][2], table[0][0][3])
        #The bottom is here for reference -- may not need it
#         doc2vec.SaveLinks()
#         #will most likely need to change this part need to change this part to a different path
#         path_to_ground_truth = '/tf/main/benchmarking/traceability/testbeds/groundtruth/english/[libest-ground-req-to-tc].txt'
#         doc2vec.MatchWithGroundTruth(path_to_ground_truth)
#         doc2vec.SaveLinks(grtruth = True)
#         #TODO find logic to LoadLink properly and display what is needed
    
    return value

In [33]:
str1 = open('test_data/RQ11.txt', 'r').read()
str2 = open('test_data/us4020.c', 'r').read()
TraceLinkValue(str1,str2,"doc2vec")

2020-11-08 14:56:18,823 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-11-08 14:56:18,838 : INFO : built Dictionary(1815 unique tokens: ['@return', 'Converts', 'The', 'a', 'and']...) from 153 documents (total 5769 corpus positions)
2020-11-08 14:56:18,840 : INFO : loading Doc2Vec object from c:\users\admin\desktop\fall2020\software engineering\project\github desktop\ds4se\ds4se\model\doc2vec_libest.model
2020-11-08 14:56:18,866 : INFO : loading vocabulary recursively from c:\users\admin\desktop\fall2020\software engineering\project\github desktop\ds4se\ds4se\model\doc2vec_libest.model.vocabulary.* with mmap=None
2020-11-08 14:56:18,869 : INFO : loading trainables recursively from c:\users\admin\desktop\fall2020\software engineering\project\github desktop\ds4se\ds4se\model\doc2vec_libest.model.trainables.* with mmap=None
2020-11-08 14:56:18,870 : INFO : loading wv recursively from c:\users\admin\desktop\fall2020\software engineering\project\github desktop\ds4se\ds4s

(31.28768539428711, 0.03097156045062732)

In [6]:
expected = (0.5401013460499512, 0.6493079189657213)

In [34]:
#FacadeTest
#Test case for proto TLV

#Prototype should print a value between 0 or 1. Input values aren't used
testTLV = TraceLinkValue("a a b", "a b c", "word2vec")
isinstance(testTLV,tuple)

2020-11-08 14:56:21,217 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-11-08 14:56:21,224 : INFO : built Dictionary(1815 unique tokens: ['@return', 'Converts', 'The', 'a', 'and']...) from 153 documents (total 5769 corpus positions)
2020-11-08 14:56:21,225 : INFO : loading Word2Vec object from c:\users\admin\desktop\fall2020\software engineering\project\github desktop\ds4se\ds4se\model\word2vec_libest.model
2020-11-08 14:56:21,493 : INFO : loading wv recursively from c:\users\admin\desktop\fall2020\software engineering\project\github desktop\ds4se\ds4se\model\word2vec_libest.model.wv.* with mmap=None
2020-11-08 14:56:21,493 : INFO : setting ignored attribute vectors_norm to None
2020-11-08 14:56:21,494 : INFO : loading vocabulary recursively from c:\users\admin\desktop\fall2020\software engineering\project\github desktop\ds4se\ds4se\model\word2vec_libest.model.vocabulary.* with mmap=None
2020-11-08 14:56:21,495 : INFO : loading trainables recursively from c:\users\a

True

In [35]:
#export
"""
Calculate the number of documents of two artifacts. Since in each dataframe, each document takes exactly one row, just counting the number 
of rows in each dataframe will gives the result. 
:param source: a dataframe of the entire source file  
:param target: a dataframe of the entire target file 
:returns: a list containing the number of documents in both source and target artifacts and the difference between the size of two artifacts
"""
def NumDoc(source, target):
    source_doc = source.shape[0]
    target_doc = target.shape[0]
    difference = source_doc - target_doc
    return [source_doc, target_doc, difference, -difference]

In [36]:
#example use
d = {'contents': ['this is a content of the source file', 'hello world', 'this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
target = {'contents': ['this is a content of the target file', 'i like banana', 'this is the content of another target file that need to be processed']}
df2 = pd.DataFrame(data=target)
NumDoc(df, df2)

[3, 3, 0, 0]

In [37]:
#FacadeTest
#Test Case for Proto NumDoc

#Prototype should print an array based on the randomly generated numbers. Input values aren't used.
datas1 = ["Form does not differ from the void, and the void does not differ from the form."]
datas2 = ["Form is the void, and the void is form."]
df1 = pd.DataFrame(data=datas1,columns=['contents'] )
df2 = pd.DataFrame(data=datas2,columns=['contents'])
testND = NumDoc(df1,df2)
assert(isinstance(testND, list))
assert(testND[0] == 1)
assert(testND[1] == 1)
assert(testND[2] == 0)
assert(testND[3] == 0)

In [38]:
#export
"""
Calculate the number of vocabulary in each of the two artifacts. This method calls the helper function that uses a bpe model.The helper
function returns a counter object of all tokens and their occurrence, the length of the list is the vocab size.
:param source: a dataframe of the entire source file  
:param target: a dataframe of the entire target file 
:returns: a list containing the vocabulary size of both source and target artifacts and the difference between the vocabulary sizes two artifacts
"""
def VocabSize(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return a list containing the the difference between the two files in terms of vocab
    source_list = preprocess(source)
    target_list = preprocess(target)
    source_size = len(source_list[0])
    target_size = len(target_list[0])
    difference = source_size - target_size
    return [source_size, target_size, difference, -difference]  

In [39]:
#example use
d = {'contents': ['this is a content of the source file', 'hello world', 'this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
target = {'contents': ['this is a content of the target file', 'i like banana', 'this is the content of another target file that need to be processed']}
df2 = pd.DataFrame(data=target)
VocabSize(df, df2)

[17, 18, -1, 1]

In [40]:
#FacadeTest
#Test case for Vocab size
datas1 = ["Form does not differ from the void, and the void does not differ from the form."]
datas2 = ["Form is the void, and the void is form."]
df1 = pd.DataFrame(data=datas1,columns=['contents'] )
df2 = pd.DataFrame(data=datas2,columns=['contents'])
#Prototype should return an array based on random number values. Inputs aren't used.
testVS = VocabSize(df1,df2)
assert(isinstance(testVS, list))

In [41]:
#export
"""
Calculate the average number of token per document in each of the two artifacts. This method calls the helper function that uses a bpe model.The helper
function returns a counter object of all tokens and their occurrence, sum up all the occurrence is the total number of tokens across entire actifact. 
Divide that number by the number of documents will get the result we need. 
:param source: a dataframe of the entire source file  
:param target: a dataframe of the entire target file 
:returns: a list containing the average number of token per document in both source and target artifacts and the difference between these two. 
"""
def AverageToken(source, target):

    #return a list containing the the difference between the two files in terms of tokens
    source_doc = source.shape[0]
    target_doc = target.shape[0]
    
    source_list = preprocess(source)
    target_list = preprocess(target)
    
    source_total_token = sum(source_list[0].values())
    target_total_token = sum(target_list[0].values())

    source_token = source_total_token/source_doc
    target_token = target_total_token/target_doc
    difference = source_token - target_token
    return [source_token, target_token, difference, -difference]

In [42]:
#example use
d = {'contents': ['this is a content of the source file', 'hello world', 'this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
target = {'contents': ['this is a content of the target file', 'i like banana', 'this is the content of another target file that need to be processed']}
df2 = pd.DataFrame(data=target)
AverageToken(df,df2)

[15.333333333333334, 16.0, -0.6666666666666661, 0.6666666666666661]

In [43]:
#FacadeTest
#Test Case for AverageToken

#Prototype should return an array/list. Inputs aren't used
testAT = AverageToken(df1,df2)
assert(isinstance(testAT, list))

In [44]:
#export
"""
Calculate the top 3 most frequent token in one artifacts. This method calls the helper function that uses a bpe model.The helper
function returns a counter object of all tokens and their occurrence, get the first three elements will gives us the most frequent
token. Divide their occurrence with totle number of tokens to get actual frequency
Divide that number by the number of documents will get the result we need. 
:param artifacts_df: a dataframe of contents that need to be processed
:returns: a dictionary of top three most frenquent token with their frenquency
"""
def Vocab(artifacts_df):
    #Note: we can add a parameter for user to specify the number of most frequent token to return
    cnts = preprocess(artifacts_df)
    vocab_list = cnts[0].most_common(3)
    total = sum(cnts[0].values())
    vocab_dict = dict()
    vocab_dict[vocab_list[0][0]] = [vocab_list[0][1], vocab_list[0][1]/total]
    vocab_dict[vocab_list[1][0]] = [vocab_list[1][1], vocab_list[1][1]/total]
    vocab_dict[vocab_list[2][0]] = [vocab_list[2][1], vocab_list[2][1]/total]

    return vocab_dict

In [45]:
#example use
d = {'contents': ['this is a content of the source file', 'hello world', 'this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
Vocab(df)

{'▁': [23, 0.5],
 'this': [2, 0.043478260869565216],
 'is': [2, 0.043478260869565216]}

In [46]:
#FacadeTest
#Test Case for Vocab

#Prototype Vocab should return dictionary type. 
#Will expand to test dictionary values once presets can be generated and tested
testVocab = Vocab(df1)

assert(isinstance(testVocab,dict))

In [50]:
  #export
"""
Calculate the top 3 most frequent token in both source and target artifacts. This method calls the helper function that uses a bpe model 
with combined dataframe.The helperfunction returns a counter object of all tokens and their occurrence, get the first three elements 
will gives us the most frequent token. Divide their occurrence with totle number of tokens to get actual frequency. 
Divide that number by the number of documents will get the result we need. 
:param source: a dataframe of the entire source file  
:param target: a dataframe of the entire target file 
:returns: a dictionary of top three most frenquent token with their frenquency
"""
def VocabShared(source, target):
    df = pd.concat([source, target])
    return Vocab(df) 

In [51]:
#example use
d = {'contents': ['this is a content of the source file', 'hello world', 'this this is the content of another source file that need to be processed']}
df = pd.DataFrame(data=d)
target = {'contents': ['this is a content of the target file', 'i like banana banana banana banana banana', 'this is the content of another target file that need to be processed']}
df2 = pd.DataFrame(data=target)
VocabShared(df, df2)

{'▁': [52, 0.5],
 'this': [5, 0.04807692307692308],
 'banana': [5, 0.04807692307692308]}

In [52]:
#FacadeTest
#Test VocabShared

#Prototype should show a dictionary
#Test will be expanded to verify frequency verification in two samples
testVocabS = VocabShared("source","target")
assert(isinstance(testVocabS,dict))


TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [23]:
#export
"""
Calculate the total number of vocabulary size of both soruce and target artifacts. The method first combines two dataframes together
and then call help function preprocess to get counter object. The length of resulting object is the total size of vocab. 
:param source: a dataframe of the entire source file  
:param target: a dataframe of the entire target file 
:returns: a number indicating the vocabulary size of both soruce and target artifacts
"""
def SharedVocabSize(source, target):
    df = pd.concat([source, target])
    df_counts = preprocess(df)
    shared_size = len(df_counts[0])
    return shared_size

In [53]:
#FacadeTest
#Test Shared vocab size

#Prototype should return an int
testSVS = SharedVocabSize("source","target")
assert(testSVS)

NameError: name 'SharedVocabSize' is not defined

In [25]:
################## Dummy Tests

In [26]:
#export

def MutualInformation(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return the mutual information 
    mutual_information = random.randint(100,200)
    return mutual_information

In [27]:
#FacadeTest
#Test Mutual information

#Proto mutual information should get an int
testMutualInfo = MutualInformation("source","target")
assert(testMutualInfo in range(100,200))

In [13]:

df2 = pd.DataFrame(data=target)
i.dit_shannon(preprocess(df)[0])
df

,contents
0,this is a content of the source file
1,hello world
2,this is the content of another source file tha...
0,this is a content of the target file
1,i like banana
2,this is the content of another target file tha...


In [14]:
#export
"""
Calculate the cross entropy of soruce and target artifacts. The method first combines two dataframes together
and then call the dit_shannon method to calcualte cross entropy with a counter object. 
:param source: a dataframe of the entire source file  
:param target: a dataframe of the entire target file 
:returns: a number indicating the vocabulary size of both soruce and target artifacts
"""
def CrossEntropy(source, target):
    #param source a dataframe of the entire source artifact  
    #param target a dataframe of the entire target artifact 
    #return the entropy
    combined = source.append(target)
    entropy = i.dit_shannon(preprocess(combined)[0])

    return entropy
    

#     cross_entropy = random.randint(100,200)#looks like it is the msi funciton in the classes for word2vec or doc2vec
#     cross_entropy = get_system_entropy_from_df(source, "col1",)
#     return cross_entropy

In [15]:
#FacadeTest
#Test Cross Entropy

#Proto cross Entropy should return an int
testCrossEntropy = CrossEntropy("Sauce","Target")
assert(testCrossEntropy in range(100,200))

#Eventually cross entropy will properly implement Entropy calculation and the test will compare a known/predicted entropy
# to a calculated value


AttributeError: 'str' object has no attribute 'append'

In [76]:
def makeArray(text):
    print("text:",text)
    return np.fromstring(text[1:-1],sep=' ')
    
#export
def KLDivergence(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return the divergence  
    
    #we are going to use the TSNE function since this preforms the divergence
    source_df = pd.DataFrame({ "ids": ["source"],  "text":[source]})
    target_df = pd.DataFrame({ "ids": ["target"],  "text":[target]})
    source_df = source_df["text"].apply(makeArray)
    target_ef = target_df["text"].apply(makeArray)
    source_df.head()
    target_df.head()
    divergence = random.randint(100,200)
    return divergence

In [77]:
#FacadeTest
#Test KLDivergence

#Proto KLDivegence will return an random int
testKLDiver = KLDivergence("source","target")
assert(testKLDiver in range(100,200))

text: source
text: target


/Users/danielquiroga/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
